In [48]:
OFF_FORM = 'ALL'
HASH = 'ALL'

In [99]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from IPython.display import HTML

In [50]:
Master = pd.read_csv('MasterDF')
Master = Master.drop(columns = {'Unnamed: 0'})

In [100]:
#This function is for displaying DFs side by side. Found online from an article. Cannot explain what it does. Just know how to call it
def side_by_side(*dfs):
    html = '<div style="display:flex">'
    for df in dfs:
        html += '<div style="margin-right: 2em">'
        html += df.to_html()
        html += '</div>'
    html += '</div>'
    display(HTML(html))

In [54]:
#THIS FUNCTION SHOWS THE RAW PLAY BY PLAY DATA
def common_filtering(form, hmark):
    #A bunch of if statements to show each different way of inputs
    if (form != 'ALL') & (hmark != 'ALL'): #Checks to see if all filters are applied
        F1 = Master[(Master.OFF_FORM == form) & (Master.HASH == hmark)] #Makes a DF of the filter
        return F1
    elif (form == 'ALL') & (hmark != 'ALL'): #Only the formations are not filtered
        F1 = Master[(Master.HASH == hmark)]
        return F1
    elif (form != 'ALL') & (hmark == 'ALL') : #Checks to see if only Form filter is set
        F1 = Master[Master.OFF_FORM == form] # Creates DF of only that form
        return F1
    else: #If no filters are set. Everything is ALL
        F1 = Master
        return F1

In [89]:
#This finds the number of times a play was called based on the 3 dropdown inputs
def PlayCount(form, hmark):
    if (form != 'ALL') & (hmark != 'ALL'): #Checks to see if all filters are applied for each 
        topplays = Master[Master.OFF_FORM == form]
        topplays = topplays[topplays.HASH == hmark]
        total = topplays.value_counts('OFF_FORM')
        total = Play_count_rename(total)
        total['GN_LS'] = topplays['GN_LS'].sum() #Can do this because its filtered all the way down
        total['AVG_YDS'] = round(total['GN_LS'] / total['PLAY_COUNT'])
        return total
    elif (form == 'ALL') & (hmark != 'ALL'): #Checks to see if only the Formation is all
        topplays = Master[Master.HASH == hmark]
        total = topplays.value_counts('OFF_FORM')
        total = Play_count_rename(total)
        t = Master.groupby('OFF_FORM')['GN_LS'].sum() #This is needed to find the total yards gained after filtered out.
        total = total.merge(t, on = 'OFF_FORM') #Merging the sum into the final display
        total['AVG_YDS'] = round(total['GN_LS'] / total['PLAY_COUNT'])
        return total
# EACH FILTER BELOW DOES THE SAME AS ABOVE FILTER. *Does not include the top filter
    elif (form != 'ALL') & (hmark == 'ALL'): #Checks to see if the hash filter is not applied
        topplays = Master[Master.OFF_FORM == form]
        total = topplays.value_counts('OFF_FORM')
        total = Play_count_rename(total)
        total['GN_LS'] = topplays['GN_LS'].sum()
        total['AVG_YDS'] = round(total['GN_LS'] / total['PLAY_COUNT'])
        return total
    elif (form == 'ALL') & (hmark != 'ALL'): #Checks to see if only the hash is applied
        topplays = Master[Master.HASH == hmark]
        topplays = topplays[topplays.HASH == hmark]
        total = topplays.value_counts('OFF_FORM')
        total = Play_count_rename(total)
        total['GN_LS'] = topplays['GN_LS'].sum()
        total['AVG_YDS'] = round(total['GN_LS'] / total['PLAY_COUNT'])
        return total
    else: #No filters applied
        total = Master.value_counts('OFF_FORM')
        total = Play_count_rename(total)
        t = Master.groupby('OFF_FORM')['GN_LS'].sum()
        total = total.merge(t, on = 'OFF_FORM')
        total['AVG_YDS'] = round(total['GN_LS'] / total['PLAY_COUNT'])
        return total

#This is just to be called to change the names of the columns. 
def Play_count_rename(DF):
    T1 = pd.DataFrame(DF).reset_index()
    T1.columns = ['OFF_FORM','PLAY_COUNT']
    return T1

In [90]:
#Finding the count of passes and runs from the filters
def Pass_Run_CT(form, hmark):
    if (form != 'ALL') & (hmark != 'ALL'): #Checks to see if all filters are applied
        df = Master[Master.OFF_FORM == form] #Next two lines filter down the data 
        df = df[df.HASH == hmark] 
        count = df.value_counts('PLAY_TYPE') # Finds the total of each pass or run was called
        df1 = R_P_df(count) #Makes the list into a dataframe
        df2 = df.groupby('PLAY_TYPE')['GN_LS'].sum() #With a new place holder, finding the total yards gained
        df1 = df1.merge(df2, on = 'PLAY_TYPE') #Merging the total yards found with the Pass or run total DataFrame
        df1['AVG_YDS'] = round(df1['GN_LS'] / df1['COUNT']) #Finding the avg yards
        return df1
#Each filter after is made the same as above. 
    elif (form == 'ALL') & (hmark != 'ALL'):
        df = Master[Master.HASH == hmark]
        count = df.value_counts('PLAY_TYPE')
        df1 = R_P_df(count)
        df2 = df.groupby('PLAY_TYPE')['GN_LS'].sum()
        df1 = df1.merge(df2, on = 'PLAY_TYPE')
        df1['AVG_YDS'] = round(df1['GN_LS'] / df1['COUNT'])
        return df1
    elif (form != 'ALL') & (hmark == 'ALL'):
        df = Master[Master.OFF_FORM == form]
        count = df.value_counts('PLAY_TYPE')
        df1 = R_P_df(count)
        df2 = df.groupby('PLAY_TYPE')['GN_LS'].sum()
        df1 = df1.merge(df2, on = 'PLAY_TYPE')
        df1['AVG_YDS'] = round(df1['GN_LS'] / df1['COUNT'])
        return df1
    else:
        df = Master.value_counts('PLAY_TYPE')
        df = R_P_df(df)
        df1 = Master.groupby('PLAY_TYPE')['GN_LS'].sum()
        df = df.merge(df1, on = 'PLAY_TYPE')
        df['AVG_YDS'] = round(df['GN_LS'] / df['COUNT'])
        return df
    
    
#Renaming columns function   
def R_P_df(DF):
    t1 = pd.DataFrame(DF).reset_index()
    t1.columns = ['PLAY_TYPE','COUNT']
    return t1

In [91]:
#Finding where plays are being called by a range yardage. 10+yds = Deep, 5-10 = mid, 1-5 = short
def yd_range(form, hmark):
    if (form != 'ALL') & (hmark != 'ALL'): #Checks to see if all filters are applied
        dd = Master[Master.OFF_FORM == form]
        dd = dd[dd.HASH == hmark]
        count = dd.groupby(['DN','DIST']).agg(COUNT =('OFF_FORM','count'))
        dd1 = dd.groupby(['DN','DIST'])['GN_LS'].sum()
        count = count.merge(dd1, on = ['DN','DIST'])
        count['AVG_YDS'] = round(count['GN_LS'] / count['COUNT'])
        return count
    elif (form == 'ALL') & (hmark != 'ALL'): #Checks to see if only the Formation is all
        dd = Master[Master.HASH == hmark]
        count = dd.groupby(['DN','DIST']).agg(COUNT =('OFF_FORM','count'))
        dd1 = dd.groupby(['DN','DIST'])['GN_LS'].sum()
        count = count.merge(dd1, on = ['DN','DIST'])
        count['AVG_YDS'] = round(count['GN_LS'] / count['COUNT'])
        return count
    elif (form != 'ALL') & (hmark == 'ALL'):
        dd = Master[Master.OFF_FORM == form]
        count = dd.groupby(['DN','DIST']).agg(COUNT =('OFF_FORM','count'))
        dd1 = dd.groupby(['DN','DIST'])['GN_LS'].sum()
        count = count.merge(dd1, on = ['DN','DIST'])
        count['AVG_YDS']  = round(count['GN_LS'] / count['COUNT'])
        return count
    else:
        dd = Master.groupby(['DN','DIST']).agg(COUNT =('OFF_FORM','count'))
        dd1 = Master.groupby(['DN','DIST'])['GN_LS'].sum()
        dd = dd.merge(dd1, on = ['DN','DIST'])
        dd['AVG_YDS'] = round(dd['GN_LS'] / dd['COUNT'])
        return dd

In [92]:
#Finding the number of times called of formation was called per down
# NEED TO ADD THE SORT BY DN TO KEEP THAT COLUMN AT 1,2,3,4 instead of moving based on the total count number
def Per_Down(form, hmark):
    if (form != 'ALL') & (hmark != 'ALL'): #Checks to see if all filters are applied
        df = Master[Master.OFF_FORM == form]
        df = df[df.HASH == hmark]
        df = df.value_counts('DN')
        df = downs(df)
        df = df.sort_values('DN')
        return df
    elif (form == 'ALL') & (hmark != 'ALL'):
        df = Master[Master.HASH == hmark]
        df = df.value_counts('DN')
        df = downs(df)
        df = df.sort_values('DN')
        return df
    elif (form != 'ALL') & (hmark == 'ALL'):
        df = Master[Master.OFF_FORM == form]
        df = df.value_counts('DN')
        df = downs(df)
        df = df.sort_values('DN')
        return df
    else:
        df = Master.value_counts('DN')
        df = downs(df)
        df = df.sort_values('DN')
        return df
    
#Renaming columns function    
def downs(DF):
    t1 = pd.DataFrame(DF).reset_index()
    t1.columns = ['DN','COUNT']
    return t1

In [93]:
RAW = common_filtering(OFF_FORM, HASH)
Pcount = PlayCount(OFF_FORM, HASH)
Ptypes = Pass_Run_CT(OFF_FORM, HASH)
Yrange = yd_range(OFF_FORM, HASH)
Downs = Per_Down(OFF_FORM, HASH)

In [111]:
side_by_side(Pcount, Ptypes, Yrange, Downs)

,OFF_FORM,PLAY_COUNT,GN_LS,AVG_YDS
0,THEODORE,164,521.0,3.0
1,ALVIN,127,735.0,6.0
2,DBLS,88,340.0,4.0
3,CLEAR,55,303.0,6.0
4,FREE,51,299.0,6.0
5,TRIPS TIGHT,23,121.0,5.0
6,LEOPARD,8,1.0,0.0
7,RABBIT,6,-7.0,-1.0
8,TIGER,3,4.0,1.0
9,LION,2,8.0,4.0


AttributeError: 'str' object has no attribute 'to_html'

/tmp/ipykernel_7900/2421223472.py:1: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  test3 = pd.concat(Test, Pcount)


TypeError: first argument must be an iterable of pandas objects, you passed an object of type "str"

In [104]:
display (RAW)

,Week,PLAY_#,QTR,SERIES,ODK,DN,DIST,HASH,YARD_LN,OFF_FORM,DECLARE,PLAY_DIR,OFF_STR,PLAY_TYPE,DEF_FRONT,MOTION_DIR,RESULT,GN_LS,EFF
0,1,6,1,1.0,O,1.0,10.0,L,45.0,NaN,NaN,L,L,Pass,3-2,R,Penalty,-5.0,N
1,1,7,1,1.0,O,1.0,15.0,L,50.0,THEODORE,NaN,L,R,Run,3-2,NaN,Rush,8.0,Y
2,1,8,1,1.0,O,2.0,7.0,L,42.0,THEODORE,NaN,R,R,Pass,3-2,NaN,Incomplete,0.0,N
3,1,9,1,1.0,O,3.0,7.0,L,42.0,THEODORE,NaN,R,R,Pass,3-2,NaN,Incomplete,0.0,N
4,1,10,1,1.0,O,4.0,7.0,L,42.0,THEODORE,NaN,L,R,Pass,3-2,NaN,Complete,9.0,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
548,10,146,4,11.0,O,2.0,10.0,L,-15.0,DBLS,NaN,L,BAL,Pass,4-1,NaN,Incomplete,0.0,N
549,10,147,4,11.0,O,3.0,10.0,L,-15.0,DBLS,NaN,L,BAL,Pass,4-1,NaN,Incomplete,0.0,N
550,10,148,4,11.0,O,4.0,10.0,L,-15.0,THEODORE,NaN,R,R,Pass,4-1,NaN,Complete,5.0,N
551,10,154,4,12.0,O,1.0,10.0,L,-20.0,FREE,NaN,L,R,Pass,4-0,NaN,Incomplete,0.0,N
